In [46]:
%load_ext autoreload
%autoreload 2

import seaborn as sns
import pandas as pd
import polars as pl
import shap

from biobank_olink.constants import PROJECT_DATA

nb_data = PROJECT_DATA / "olink"

sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
index_cols = ["sex", "age", "BMI", "Smokinstatus", "HTNgroup", "ASI"]
cols = ["protein_id", "result"] + index_cols

coding = pl.read_csv(nb_data / "coding143.tsv", separator="\t").with_columns(
    pl.col("meaning").str.split(";").list.get(0).alias("meaning"),
)
df = (
    pl.scan_parquet(nb_data / "Olink whole dataset long format ASI_*.parquet")
    .select(cols)
    .filter(pl.col("ASI").is_not_null(), pl.col("HTNgroup") != 2)
    .with_columns(
        pl.col("protein_id").replace_strict(coding["coding"], coding["meaning"],
                                            return_dtype=pl.String)
    )
).collect()

df = df.pivot(on="protein_id", index=index_cols, values="result")

In [23]:
df_htn = df.filter(pl.col("HTNgroup") == 1).select(pl.exclude("HTNgroup")).to_pandas()
df_ntn = df.filter(pl.col("HTNgroup") == 0).select(pl.exclude("HTNgroup")).to_pandas()

"htn: {}, ntn: {}".format(df_htn.shape, df_ntn.shape)

'htn: (2174, 2928), ntn: (3053, 2928)'

In [69]:
import numpy as np

from xgboost import XGBRegressor


def impute_and_standardize(_df):
    _df = _df.fillna(_df.mean())
    return (_df - _df.mean()) / _df.std()


def shap_to_importance(values):
    return pd.Series(np.abs(values.values).sum(axis=0), index=x.columns).sort_values(
        ascending=False)


def train_xgb(x, y, n_estimators=40):
    estimator = XGBRegressor(n_estimators=n_estimators, tree_method="hist", random_state=42)
    estimator.fit(x, y)
    print("R2:", estimator.score(x, y))
    return estimator


## XGBoost

In [70]:
x, y = df_ntn.drop("ASI", axis=1), df_ntn["ASI"]

model = train_xgb(x, y)

R2: 0.9761645263699109


In [71]:
explainer = shap.TreeExplainer(model)
shap_values = explainer(x)
# shap.summary_plot(shap_values, x, max_display=10)
shap_to_importance(shap_values)

age        1323.922241
SPINK6      899.275452
PRCP        355.819122
sex         327.657654
CXCL17      325.735229
              ...     
IGFBP6        0.000000
IGFBP3        0.000000
IGFBP2        0.000000
IGF2BP3       0.000000
SOWAHA        0.000000
Length: 2927, dtype: float32

In [72]:
x, y = df_htn.drop("ASI", axis=1), df_htn["ASI"]

model = train_xgb(x, y)

R2: 0.9899182658174113


In [74]:
explainer = shap.TreeExplainer(model)
shap_values = explainer(x)
# shap.summary_plot(shap_values, x, max_display=10)
shap_to_importance(shap_values)


SPINK6    750.584595
GDF15     264.549805
age       252.890915
CDHR2     226.077393
PGF       216.860565
             ...    
SPRED2      0.000000
BEX3        0.000000
MDM1        0.000000
MECR        0.000000
SRPX        0.000000
Length: 2927, dtype: float32

In [75]:
pd.read_excel(nb_data / "GLM Olink ASI ntn Zscore 0.1.xlsx", skiprows=1)

,Zresult,protein_id,protein_name,B,Std.E,t,Sig.,Rank,Sig.-adj FDR,Lower bound,Upper bound,Squared,Parameter,Power,number_subjects
0,Zresult,2541.0,SPINK6;Serine protease inhibitor Kazal-type 6,0.360215,0.040518,8.890359,0.0,1,2.442627e-15,0.280783,0.439647,0.015528,8.890359,1.0,5023.0
1,Zresult,1357.0,IGSF9;Protein turtle homolog A,0.381995,0.052664,7.253424,0.0,2,6.984484e-10,0.278747,0.485243,0.011832,7.253424,1.0,4406.0
2,Zresult,2128.0,PRAP1;Proline-rich acidic protein 1,0.330221,0.049485,6.673095,0.0,3,2.743049e-08,0.233205,0.427237,0.010021,6.673095,0.999999,4411.0
3,Zresult,1189.0,GNPDA1;Glucosamine-6-phosphate isomerase 1,0.279562,0.041979,6.659536,0.0,4,2.268915e-08,0.19726,0.361864,0.010538,6.659536,0.999999,4176.0
4,Zresult,534.0,CES1;Liver carboxylesterase 1,0.307877,0.047922,6.424501,0.0,5,8.463242e-08,0.213928,0.401826,0.008304,6.424501,0.999996,4941.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,Zresult,1773.0,"MTIF3;Translation initiation factor IF-3, mitochondrial",-0.000035,0.043857,-0.000803,0.999359,2921,1.000044e+00,-0.086017,0.085947,0.0,0.000803,0.05,4342.0
2921,Zresult,2570.0,"ST8SIA1;Alpha-N-acetylneuraminide alpha-2,8-sialyltransferase",-0.000009,0.041118,-0.000209,0.999833,2922,1.000175e+00,-0.080622,0.080605,0.0,0.000209,0.05,4025.0
2922,Parameter,NaN,NaN,B,Std. Error,t,Sig.,2923,NaN,95% Confidence Interval,NaN,Partial Eta Squared,Noncent. Parameter,Observed Powerb,NaN
2923,Zresult,1173.0,GLIPR1;Glioma pathogenesis-related protein 1,-4.450034,NaN,NaN,NaN,2924,0.000000e+00,NaN,NaN,NaN,NaN,NaN,11.0
